<a href="https://colab.research.google.com/github/meisamgh/Machine_Learning_Trading/blob/main/Alpha_Factor_Anlysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib
!pip install alphalens

In [ ]:
import pandas_datareader as web
tickers = ['AAPL', 'A' , "AA" , 'AAL' , 'AAMC' , 'AAN' , 'AAOI' , 'AAON' , 'AAP' , 'AAT']
data = web.DataReader(tickers , 'yahoo', start='2018/01/01', end='2019/12/31')
#data.columns = ['Adj close' , 'Close' , 'High' , 'Low' , 'Open' , 'Volume']

/usr/local/lib/python3.7/dist-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'AAPL', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/usr/local/lib/python3.7/dist-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'A', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/usr/local/lib/python3.7/dist-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'AA', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/usr/local/lib/python3.7/dist-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'AAL', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/usr/local/lib/python3.7/dist-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'AAMC', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/usr/local/lib/python3.7/dist-packages/pandas_datareader/base.py:272: SymbolWarning: F

RemoteDataError: ignored

In [ ]:
data = data.stack('Symbols')

In [ ]:
data.index.names = ['date','ticker']
data.columns = ['adj_close' , 'close' , 'high' , 'low' , 'open' , 'volume']
ohlcv = ['open', 'high', 'low', 'close', 'volume']
open = data.open
high = data.open
low = data.low.unstack('ticker')
close = data.close.unstack('ticker')
volume = data.volume.unstack('ticker')
#vwap = o.add(h).add(l).add(c).div(4)
#adv20 = v.rolling(20).mean()
data['returns'] = data.close.pct_change()
data['ret_fwd'] = data.close.shift(-1)
data.dropna( 0 , inplace = True)
r = data.returns.unstack('ticker')


In [ ]:
import pandas as pd 
import numpy as np
from talib import RSI , BBANDS , ATR
import talib

In [ ]:
data.head()

In [ ]:
def MACDEXT(data ,  fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0):
    
    return  talib. MACDEXT(data.close, fastperiod , fastmatype, slowperiod , slowmatype, signalperiod, signalmatype)

In [ ]:
factor = pd.DataFrame()
factor['rsi'] = data.groupby(level='ticker').apply(AROON)

In [ ]:
factor = pd.DataFrame()
prices = data 
#factor['dollar_vol'] = prices.loc[:, 'close'].mul(prices.loc[:, 'volume'], axis=0)
#factor['dollar_vol'] = (prices.groupby('ticker', group_keys=False, as_index=False)).dollar_vol.rolling(window=21).mean().reset_index(level=0, drop=True)
#factor['dollar_vol_rank'] = (prices.groupby('date') .dollar_vol .rank(ascending=False))

factor['rsi'] = prices.groupby(level='ticker').close.apply(RSI)

def compute_bb(close):
  high, mid, low = BBANDS(close) 
  return pd.DataFrame({'bb_high': high, 'bb_low': low}, index=close.index)

prices = (prices.join(prices.groupby(level='ticker') .close .apply(compute_bb)))

factor['bb_high'] = prices.bb_high.sub(prices.close).div(prices.bb_high). apply(np.log1p) 
factor['bb_low'] = prices.close.sub(prices.bb_low).div(prices.close). apply(np.log1p)

def compute_atr(stock_data):
  df = ATR(stock_data.high, stock_data.low, stock_data.close, timeperiod=14)
  return df.sub(df.mean()).div(df.std()) 

factor['atr'] = (prices.groupby('ticker', group_keys=False) .apply(compute_atr))

from alphalens.tears import create_full_tear_sheet
create_full_tear_sheet(alphalens_data)


In [ ]:
pp = prices.close.unstack()

In [ ]:
rsi = factor[['rsi']]
atr = factor[['atr']]

bb_low = factor[['bb_low']]
bb_high = factor[['bb_high']]


In [ ]:
from alphalens.utils import get_clean_factor_and_forward_returns , get_clean_factor , compute_forward_returns

fw_return = compute_forward_returns(factor, pp, periods=(1, 5, 10), filter_zscore=None)
al_data = get_clean_factor(factor = rsi , forward_returns= fw_return  , quantiles=5  )

In [ ]:
from alphalens.utils import get_clean_factor_and_forward_returns , get_clean_factor
from alphalens.performance import *
from alphalens.plotting import *
from alphalens.tears import *

import seaborn as sns
import matplotlib.pyplot as plt

HOLDING_PERIODS = (5, 10, 21, 42) 
QUANTILES = 5 
alphalens_data = get_clean_factor_and_forward_returns(factor=rsi, prices =  pp , periods=HOLDING_PERIODS, quantiles=QUANTILES)

In [ ]:
def generate_factor_stats(results , prices ,factors):
    factor_stats = {}
    for name, factor in factors.items():
        facor_data = alphalens.utils.get_clean_factor_and_forward_returns(
            factor = results[name], 
            prices = prices, 
            periods = (1,5,21),
            max_loss= 0.95)
        ic = alphalens.performance.factor_information_coefficient()
        risk_adjusted_ic = (ic.mean()/ic.std()).round(5)['21D']
        
        returns = alphalens.performance.factor_returns(facor_data)
        factor_stats[name] = [risk_adjusted_ic , returns]
        
    return factor_stats

In [ ]:
!pip install zipline

In [ ]:
from zipline.pipeline import Pipeline, CustomFactor
from talib import ADX 


import pandas_datareader as web
tickers = ['AAPL', 'A' , "AA" , 'AAL' , 'AAMC' , 'AAN' , 'AAOI' , 'AAON' , 'AAP' , 'AAT']
data = web.DataReader(tickers , 'yahoo', start='2018/01/01', end='2019/12/31')


In [ ]:
class ADX(CustomFactor):
    inputs = [data.high , data.low , data.close]
    window_length = 1
    
    def compute (self, today , assets , out, high , low ,close):
        out[:] = talib.ADX(high[-1], low[-1], 
                           close[-1])

def compute_factors():
    adx = ADX()
    return Pipeline(columns = {'adx' : adx})